<a href="https://colab.research.google.com/github/altsoph/plays_remarker/blob/main/Remarker_generate_author_remarks_for_your_play_dialog_using_ruGPT_and_HF_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Remarker 
## Generate author remarks for your play dialog using ruGPT and HF transformers!

## Made by altsoph, feel free to reach me:
## https://twitter.com/altsoph
## https://t.me/altsoph
## http://altsoph.com

## Install env

In [1]:
!pip install torch==1.4.0
!pip3 install transformers==3.5.0

     |████████████████████████████████| 753.4MB 21kB/s 
ERROR: torchvision 0.10.0+cu102 has requirement torch==1.9.0, but you'll have torch 1.4.0 which is incompatible.
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
     |████████████████████████████████| 1.3MB 8.5MB/s 
     |████████████████████████████████| 1.1MB 50.3MB/s 
     |████████████████████████████████| 901kB 44.7MB/s 
     |████████████████████████████████| 2.9MB 45.7MB/s 


In [2]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 619, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 619 (delta 69), reused 95 (delta 59), pack-reused 505
Receiving objects: 100% (619/619), 366.72 KiB | 1.95 MiB/s, done.
Resolving deltas: 100% (369/369), done.


In [3]:
!mkdir models/

## Download files

In [4]:
!wget -O data.txt https://drive.google.com/u/0/uc?id=1QxbdBCJ66TJWRazuDCfmiIGt4mMpuzGr&export=download
!head data.txt

--2021-07-02 09:57:47--  https://drive.google.com/u/0/uc?id=1QxbdBCJ66TJWRazuDCfmiIGt4mMpuzGr
Resolving drive.google.com (drive.google.com)... 142.251.2.113, 142.251.2.139, 142.251.2.102, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-20-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/81fneclsp71ri1k6b13oi9sje34lo0ne/1625219850000/18029368614449056196/*/1QxbdBCJ66TJWRazuDCfmiIGt4mMpuzGr [following]
--2021-07-02 09:57:47--  https://doc-0c-20-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/81fneclsp71ri1k6b13oi9sje34lo0ne/1625219850000/18029368614449056196/*/1QxbdBCJ66TJWRazuDCfmiIGt4mMpuzGr
Resolving doc-0c-20-docs.googleusercontent.com (doc-0c-20-docs.googleusercontent.com)... 142.251.2.132, 2607:f8b0:4023:c0d::84
Connecting to doc-0c-20-docs.googleusercontent.com (doc-0c-20-docs.googleusercontent.com)|142.251

## Train 
The following code download our model and tokenizer from huggingface and finetune model for generating remarks.

This took around 5 minutes, stay strong!

In [5]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/remarks \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=data.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 1 \
    --block_size 2048 \
    --overwrite_output_dir

2021-07-02 09:58:08.825934: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
07/02/2021 09:58:11 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
07/02/2021 09:58:11 - INFO - filelock -   Lock 140249392254160 acquired on /root/.cache/torch/transformers/06f48b6b3173390d047e15d691fda67ae4ea7733a5eea4b6e0115f5099c4e700.b5cdfa39c63384f94159c36bc9042660c747cea5cf520b43d543bd2c68b3164d.lock
Downloading: 100% 608/608 [00:00<00:00, 598kB/s]
07/02/2021 09:58:11 - INFO - filelock -   Lock 140249392254160 released on /root/.cache/torch/transformers/06f48b6b3173390d047e15d691fda67ae4ea7733a5eea4b6e0115f5099c4e700.b5cdfa39c63384f94159c36bc9042660c747cea5cf520b43d543bd2c68b3164d.lock
07/02/2021 09:58:12 - INFO - filelock -   Lock 140249392339088 acquired on /root/.cache/torch/transformers/1b36eeb1fd7b3a6ec11bf46bde2c38e7e68f71ec774694b9e886c86001aab35d.c483bc344

## Specify an input dialog, one utterance by line:


In [6]:
input_text = """
Ты когда-нибудь слушал тишину, Ежик?	
Слушал.	
И что?	
А ничего. Тихо.	
А я люблю, когда в тишине что-нибудь шевелится.	
Приведи пример.	
Ну, например, гром.
"""

### Some code

In [12]:
import numpy as np
import torch
import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer
transformers.logging.set_verbosity(transformers.logging.ERROR)

MODEL_DIR = "models/remarks"
tok = GPT2Tokenizer.from_pretrained(MODEL_DIR)
model = GPT2LMHeadModel.from_pretrained(MODEL_DIR).cuda()

def get_and_decorate_remark(utt):
  text = f"<s>{utt}<s>"
  inpt = tok.encode(text, return_tensors="pt")
  out = model.generate(inpt.cuda(), max_length=100, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1, eos_token_id=None)

  raw_res = tok.decode(out[0])
  remark = None
  _, utterance, remark_raw = raw_res.split('<s>',2)
  if '[' in remark_raw:
    _, remark_raw = remark_raw.split('[',1)
    if ']' in remark_raw:
      remark, _ = remark_raw.split(']',1)
  if remark:
    return f'[{remark}] {utt}'
  return f'{utt}'

def render_remarks(input_text):
  for line in input_text.strip().split('\n'):
    print(get_and_decorate_remark(line))

## Run to generate remarks:
### (Repeat if you want another results)

In [14]:
render_remarks(input_text)

[задумчиво] Ты когда-нибудь слушал тишину, Ежик?	
[не сразу] Слушал.	
[встает] И что?	
[снова] А ничего. Тихо.	
[смущенно] А я люблю, когда в тишине что-нибудь шевелится.	
[неожиданно] Приведи пример.	
[смеется] Ну, например, гром.
